In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [4]:
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [5]:
import csv
import os
import numpy as np
import pandas as pd
import pickle

In [6]:
df = pd.read_csv('C:/Users/octanet/Desktop/analyse comportementale/coordonnées.csv')


In [7]:
df


,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,walk,0.404274,0.219890,-0.382095,0.999914,0.407091,0.193314,-0.347964,0.999902,0.411433,...,0.001808,0.0,0.429997,0.206316,0.022190,0.0,0.431711,0.203532,0.023668,0.0
1,walk,0.410773,0.219847,-0.330058,0.999912,0.411452,0.194058,-0.296606,0.999906,0.415504,...,0.002311,0.0,0.429815,0.203590,0.022514,0.0,0.431498,0.200955,0.024007,0.0
2,walk,0.420573,0.226272,-0.307716,0.999908,0.423206,0.202070,-0.273308,0.999909,0.427901,...,0.000236,0.0,0.440145,0.203675,0.018633,0.0,0.441526,0.201152,0.019834,0.0
3,walk,0.423506,0.227172,-0.334762,0.999890,0.425945,0.203721,-0.298645,0.999901,0.431279,...,0.001317,0.0,0.442785,0.205518,0.018848,0.0,0.444122,0.203065,0.020010,0.0
4,walk,0.425527,0.227067,-0.348007,0.999890,0.428010,0.203368,-0.312045,0.999905,0.432177,...,0.001354,0.0,0.441878,0.206097,0.019332,0.0,0.442978,0.204003,0.020518,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,use computer,0.481536,0.156910,-0.116922,0.996652,0.486159,0.099797,-0.159178,0.998400,0.496974,...,-0.014909,0.0,0.533768,0.096248,-0.023595,0.0,0.536049,0.090272,-0.025128,0.0
2029,use computer,0.478456,0.071927,-0.056967,0.997587,0.492824,0.020770,-0.098876,0.999081,0.504939,...,-0.015638,0.0,0.519282,0.005272,-0.028103,0.0,0.520472,-0.002437,-0.029993,0.0
2030,use computer,0.474017,0.096588,-0.029121,0.997750,0.476181,0.047435,-0.075084,0.999161,0.484160,...,-0.013913,0.0,0.509960,0.002182,-0.019317,0.0,0.510878,-0.004293,-0.020473,0.0
2031,use computer,0.472169,0.097775,-0.019781,0.997734,0.473138,0.048412,-0.067343,0.999168,0.480778,...,-0.014459,0.0,0.510085,0.001008,-0.021296,0.0,0.510997,-0.006190,-0.022604,0.0


In [10]:
with open('C:/Users/octanet/Desktop/analyse comportementale/pose_model.pkl','rb') as f:
    
    model = pickle.load(f)

In [12]:
cap = cv2.VideoCapture(0)
classes = []
classFile = "C:/Users/octanet/Desktop/analyse comportementale/coco_names"
with open(classFile,'rt') as f :
    classes = f.read().rstrip('\n').split('\n')
    
configPath = "C:/Users/octanet/Desktop/analyse comportementale/conf2.pbtxt"
weightsPath = "C:/Users/octanet/Desktop/analyse comportementale/yolov3 (2).weights"
net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        height,width,cT = frame.shape
        
        
        blob = cv2.dnn.blobFromImage(frame, 1/255,(320,320),(0,0,0), swapRB=True,crop=False)
        net.setInput(blob)
        layer_names = net.getLayerNames()
        output_names = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
        outputs = net.forward(output_names)
         # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
        
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))

        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
         
                    mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        
        try:
            bbox=[]
            confidences =[]
            class_ids = []
            for output in outputs:
                for detection in output:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(detection[0]* width)
                        center_y = int(detection[1]* height)
                        w =  int(detection[2]* width)
                        h= int(detection[3]* height)

                        x = int(center_x -w/2)
                        y = int(center_y -h/2)

                        bbox.append([x,y,w,h])
                        confidences.append((float(confidence)))
                        class_ids.append(class_id)
            indexes = cv2.dnn.NMSBoxes(bbox,confidences,0.5,0.3)

            if len(indexes) > 0:
                for i in indexes.flatten():
                    x,y,w,h = bbox[i]
                    label = str(classes[class_ids[i]])
            
            pose = results.pose_landmarks.landmark
            pose_row=list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

            face = results.face_landmarks.landmark
            face_row=list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())

            row = pose_row + face_row
            
            
            X=pd.DataFrame([row])
            class_action = model.predict(X)[0]
            class_action_prob = model.predict_proba(X)[0]
            #print(class_action,class_action_prob)
            
            coords = tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                                results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y,
                                                )), [640,480]).astype(int))
            
            
            
            if class_action != 'phone call' and class_action != 'use phone' and class_action != 'drink' and class_action != 'use computer' and class_action != 'eat' :
                cv2.rectangle(image, (coords[0],coords[1]+5), (coords[0]+ len(class_action)*20 , coords[1]-30),
                         (245,117,16), -1)
                cv2.putText(image, class_action , coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2 ,cv2.LINE_AA)
            
                cv2.putText(image,class_action.split('')[0],
                       (95,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
                cv2.putText(image,'prob',
                       (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
                cv2.putText(image,str(round(class_action_prob[np.argmax(class_action_prob)],2)),
                       (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            elif class_action == 'phone call' or class_action == 'use phone':
                if label == 'cell phone':
                    cv2.rectangle(image, (coords[0],coords[1]+5), (coords[0]+ len(class_action)*20 , coords[1]-30),
                         (245,117,16), -1)
                    cv2.putText(image, class_action , coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2 ,cv2.LINE_AA)
            
                    cv2.putText(image,class_action.split('')[0],
                       (95,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
                    cv2.putText(image,'prob',
                       (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
                    cv2.putText(image,str(round(class_action_prob[np.argmax(class_action_prob)],2)),
                       (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
                
             
            elif class_action == 'drink':
                if label in ['bottle','cup','wine glass']:
                    cv2.rectangle(image, (coords[0],coords[1]+5), (coords[0]+ len(class_action)*20 , coords[1]-30),
                         (245,117,16), -1)
                
                    cv2.putText(image, class_action , coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2 ,cv2.LINE_AA)
            
                    #cv2.rectangle(image,(0,0),(250,60),(245,177,16),-1)
                    #cv2.putText(image,'class',
                       #(95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
                    cv2.putText(image,class_action.split('')[0],
                       (95,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
                    cv2.putText(image,'prob',
                       (15,12),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)
                    cv2.putText(image,str(round(class_action_prob[np.argmax(class_action_prob)],2)),
                       (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            elif class_action == 'eat':
                if label in ['banana' ,'apple','sandwich','orange','broccoli','carrot','hot' ,'dog','pizza','cake']:
                    cv2.rectangle(image, (coords[0],coords[1]+5), (coords[0]+ len(class_action)*20 , coords[1]-30),
                         (245,117,16), -1)
                
                    cv2.putText(image, class_action , coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2 ,cv2.LINE_AA)
    
                    cv2.putText(image,class_action.split('')[0],
                       (95,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
                    cv2.putText(image,'prob',
                       (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
                    cv2.putText(image,str(round(class_action_prob[np.argmax(class_action_prob)],2)),
                       (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)        
               
            elif class_action == 'use computer':
                if label in ['tvmonitor','laptop','mouse','keyboard']:
                    cv2.rectangle(image, (coords[0],coords[1]+5), (coords[0]+ len(class_action)*20 , coords[1]-30),
                         (245,117,16), -1)
                    cv2.putText(image, class_action , coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2 ,cv2.LINE_AA)
            
                    cv2.putText(image,class_action.split('')[0],
                       (95,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
                    cv2.putText(image,'prob',
                       (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
                    cv2.putText(image,str(round(class_action_prob[np.argmax(class_action_prob)],2)),
                       (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
              
                    
        except:
            pass
            
        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()                        
cv2.destroyAllWindows() 